In [ ]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

# Libraries and functions

In [4]:
import numpy as np
import pandas as pd
import pandasql as ps
from pandas import Series, DataFrame
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib as mpl
import sys
import time
import math
from tqdm import tqdm
from importlib import reload
from os.path import join, normpath
from bs4 import BeautifulSoup
import os
import roamability as rb
import re

downloads = 'C:/Users/balob/Downloads'

#### To get the path

In [95]:
from pathlib import Path
path_dir = Path.cwd()
print(path_dir)
path_dir

C:\Users\balob\Documents\GITLAB\RB_BD


WindowsPath('C:/Users/balob/Documents/GITLAB/RB_BD')

In [3]:
pwd=!echo %cd%
pwd=str(pwd)
pwd=r"c:\Users\balob\Documents\GITLAB\NOC_Roamability_com\noc\noc"
pwd.replace('\\','/',100)

'c:/Users/balob/Documents/GITLAB/NOC_Roamability_com/noc/noc'

In [99]:
%reset -f

# Examples

### Soap API OCS Portal

In [1]:
%reset -f
from zeep import Client
from zeep.wsse.username import UsernameToken

connect = input("Choose connection (prod or lab):")

if connect == 'lab':
    user = 'lab_api@roamability.com'
    user = 'lab_ocs@justworksmobile.com'
    password = '25D55AD283AA400AF464C76D713C07AD' #Lab
    #api_link = 'https://172.20.39.7:8585/ocsapi/roamability/api/roamability.wsdl' #Lab
    api_link = 'https://labocsapi.roamability.com:6443/ocsapi/roamability/api/roamability.wsdl'
elif connect == 'prod':
    user = 'prod_api@roamability.com'
    password = '25D55AD283AA400AF464C76D713C07AD'
    api_link = 'http://172.20.35.12:8585/ocsapi/roamability/api/roamability.wsdl'

user_name_token = UsernameToken(user, password)
user_name_token.use_digest = True
client = Client(api_link, wsse=user_name_token)

reseller = client.service.getResellerInfo()
accounts = client.service.getAccounts()

if reseller.result.code == '1':
    print('You are connected to Reseller: {}.\
    \nAvailable credit for the Reseller: {:.0f} USD.\n'.\
    format(reseller.reseller.resellerName,reseller.reseller.availableCredit))
else:
    print('Error in getting Reseller info: {}'.format(reseller.result.description))

if accounts.result.code == '1':
    accounts_dict = {account.accountId:account.accountName for account in accounts.accounts.accounts}
    print('The Reseller contains the following accounts:')
    for i,[account_id,account_name] in enumerate(accounts_dict.items()):
        print(' {:<5}{} (accountID: {})'.format(i,account_name,account_id))
else:
    print('Error in getting Account info: {}'.format(accounts.result.description))
    
# Subscriber in lab in STI account
#subscriber_id = client.service.getSubscriberById('934311')
#subscriber_iccid = client.service.getSubscriberByICCID('8997219121000031446')
#subscriber_id

Choose connection (prod or lab):lab
You are connected to Reseller: Justworks Mobile.    
Available credit for the Reseller: 50 USD.

The Reseller contains the following accounts:
 0    Justworks Mobile test SIMs (accountID: 352598)


### Send MSU

In [508]:
from bs4 import BeautifulSoup
import roamability as rb
#reload(rb)

switch_01_payload = '24027000001f0d00010000bfff0100000000000010554f212dbbad49a1caf2573e1a626139'
switch_02_payload = '24027000001F0D00010000BFFF0100000000000040a740AD174703383798A7A6545BB36491'

#515030190000000
#1d46acf860780
add_imsi_03 = '3C02700000370d00210000bfff0100000000000040939360c23f417f147557b6940c91ba8b7e54be1c8c950a6ed1235ea8e3d70e9c2d7c34876abd68ec'


ogt = {'p4':'48790993070',
       'partner':'97254120624',
       'sure':'447797706411',
       'tot':'66893773228',
       'porto_seguro':'550549900000',
       'rusec_rus':'79028710069',
       'rusec_int':'417999880000024',
       'multi_byte_sponsor':'85263347864',
       'smart1':'639180009880',
       'smart2':'639180009881',
       'smart3':'639180009882',
       'smart4':'639180009883',
       'telzar':'972559900040',}

rb.greetings_func()
#rb.print_imsi_prof(ogt, imsi, msc, 'as_resp') #'as_mo'
#soup = BeautifulSoup(rb.sri4sm(ogt, msisdn),'xml') #250359000007598 79028710043
#soup = BeautifulSoup(rb.prn(ogt, dgt, imsi),'xml')
#soup = BeautifulSoup(rb.sai(ogt, dgt, imsi, node),'xml') # 0 - vlr; 16 - mme
#soup = BeautifulSoup(rb.sendSMS(ogt, imsi, msc, rb.get_info),'xml')
#soup = BeautifulSoup(rb.cl(ogt, dgt, imsi, d_ssn=7),'xml')
#soup

Greetings from Roamability!!! 



In [510]:
rb.greetings_func()
soup = BeautifulSoup(rb.sai(ogt['partner'], '972549620012610', '425019620012610', 0),'xml') # 0 - vlr; 16 - mme
soup

Greetings from Roamability!!! 



<?xml version="1.0" encoding="utf-8"?>
<response>
<quintupletList>
<sequence>
<rand>b4dae2566b85fe29274470e9c1117e47</rand>
<xres>9936f0d66274b189</xres>
<ck>7d68a66310afa79a0581625e37b149c9</ck>
<ik>2d90485ab98ec2fb95d0de90e0abbe0e</ik>
<autn>50e9b31c28a910005977025a71c1a20c</autn>
</sequence>
</quintupletList>
<result>0</result>
<src_gt>97254120624</src_gt>
</response>

In [70]:
import http
import re
from bs4 import BeautifulSoup
import sys

ss7_url = "172.18.11.10"
ss7_path = "/cgi-bin/ss7gw.fcgi"

def executeHTTP(request, url, path):
    client = http.client.HTTPConnection(url)
    client.request("POST", path, request, {"Content-Type": "text/xml"})
    resp = client.getresponse()
    return resp.read()

def sri4sm(ogt, msisdn, dgt):
    req = """<?xml version=\"1.0\"?>
        <ss7gw request=\"SRI_SM\">
        <d_ssn>6</d_ssn>
        <o_ssn>8</o_ssn>
        <sccp_np>7</sccp_np>
        <o_gt>%s</o_gt>
        <d_gt>%s</d_gt>
        <msisdn>%s</msisdn>
        <priority>1</priority>
        <address>%s</address>
        </ss7gw>""" % (ogt, dgt, msisdn, ogt)

    resp = executeHTTP(req, ss7_url, ss7_path)
    return resp
soup = BeautifulSoup(sri4sm('972559900040', '972559642417', '972559912014336'),'xml') #250359000007598 79028710043
soup

<?xml version="1.0" encoding="utf-8"?>
<response>
<result>-1</result>
<error>Error: Unknown Subscriber(1)</error>
<src_gt>972559900071</src_gt>
</response>

In [24]:
s = '44077FF610027100000B0ABFFF0100000000000000 '
#s = '00077FF63A02710000350ABFFF0100000000000000FFFFFFF00001070152F0100000008A567809437451000008495210693100505801082906601094005058'
#s = '00077FF63A02710000350ABFFF0100000000000000FFFFFFF00001070152F0100000008A567809437451000008495210693100505801082906601094005058'
s = s.upper()
imsi_list = re.findall('(0\d08)(\d{16})', s)
print(imsi_list)

print(app_ver)
mccmnc_extract = re.findall('FFFFFFF000\d{6}(.{6})', s)

print('This SIM card contains the following IMSIs:')
for slot, imsi in imsi_list:
    print(f"Slot {slot[1]} {''.join([imsi[i] for i in [0, 3, 2, 5, 4, 7, 6, 9, 8, 11, 10, 13, 12, 15, 14]])}")
app_ver = re.findall('FFFFFFF000(\d{6})', s)
if app_ver:
    print(f'Applet ver.: {int(app_ver[0][:2])}.{int(app_ver[0][2:4])}.{int(app_ver[0][4:6])}')
if mccmnc_extract:
    mccmnc = ''.join([mccmnc_extract[0][i] for i in [1, 0, 3, 5, 4]])
    print(f'MCCMNC: {mccmnc}')

[]
[]
This SIM card contains the following IMSIs:


 ### Format output

In [ ]:
import datetime as dt
print( 'start: {} {} :end'.format('one', 'two') ) #start: one two :end
print( 'start: {1} {0} :end'.format('one', 'two') ) #start: two one :end
print( 'start: test {:10} :end'.format('test') ) #start: test test       :end
print( 'start: test {:10} :end'.format(10) ) #start: test         10 :end
print( 'start: test {:>10} :end'.format('test') ) #start: test       test :end
print( 'start: test {:<10} :end'.format('test') ) #start: test test       :end
print( 'start: test {:_<10} :end'.format('test') ) #start: test test______ :end
print( 'start: test {:^10} :end'.format('test') ) #start: test    test    :end
print( 'start: test {:.10} :end'.format('test1test2test3') ) #start: test test1test2 :end
print( 'start: test {:_^10.5} :end'.format('test1test2test3') ) #start: test __test1___ :end
print( 'start: test {:d} :end'.format(42) ) #start: test 42 :end
print( 'start: test {:f} :end'.format(42) ) #start: test 42.000000 :end
print( 'start: test {:06.2f} :end'.format(3.141592) ) #start: test 003.14 :end
print( 'start: test {p[first]} {p[last]} :end'.format(p={'first': 'Jean-Luc', 'last': 'Picard'}) ) #start: test Jean-Luc Picard :end
print( 'start: test {p[4]} {p[5]} :end'.format(p=[4, 8, 15, 16, 23, 42]) ) #start: test 23 42 :end
print( 'start: test {:%Y-%m-%d %H:%M} :end'.format(dt.datetime(2001, 2, 3, 4, 5)) ) #start: test 2001-02-03 04:05 :end

### Search in files by Tags

In [59]:
import roamability as rb
reload(rb)

# Где искать файлы .txt с тегами
paths = ['c:/W_DATA_ROAM/ДОКУМЕНТАЦИЯ/NOC_HOW_IT_WORKS/',
        'c:/W_DATA_ROAM/МЕТОДИКИ/',
        'c:/W_DATA_ROAM/ПРОЕКТЫ/',
        'c:/W_DATA_ROAM/ДОКУМЕНТАЦИЯ/NOC_KNOWN_ISSUES/',
        'c:/W_DATA_ROAM/МЕТОДИКИ/_NOC_M_NEW_CONNECTION/',
        'c:/W_DATA_ROAM/ALARMS/',
        'c:/W_DATA_ROAM/ДОКУМЕНТАЦИЯ/PRICES',
        ]

# Если хотя бы один из тэгов присутствует. Нужно указать минимум один.
any_tags  = ['#epg_principle_of_operation']

# Если все теги присутствуют. Нужно оставить лист пустым [] если нет обязательных тэгов.
all_tags = []

rb.find_files_by_tags(paths, all_tags, any_tags)

****************************************************************************************************
c:\W_DATA_ROAM\ДОКУМЕНТАЦИЯ\NOC_HOW_IT_WORKS\EPG\EPG_readme.txt
----------------------------------------------------------------------------------------------------
#epg_principle_of_operation
#proc_epg_troubleshooting
------------------------------
Модель конфигуарции EPG.
Документация по EPG: EPG 1.2 Virtual
(Dudi, WA, 07.02.2019)
Пример блокировки трафика на EPG во время проблемы с IP.

****************************************************************************************************
c:\W_DATA_ROAM\ДОКУМЕНТАЦИЯ\NOC_HOW_IT_WORKS\EPG_troubleshooting\EPG_troubleshooting.txt
----------------------------------------------------------------------------------------------------
#proc_epg_troubleshooting
#epg_principle_of_operation
------------------------------
Использовать виртуальные машины EPG (10.101.167.5 & 10.100.167.5)
Проверка резолва APN
dig @192.118.11.184 ultradata.apn.epc.mnc00

### Search in ALARMS by folder names

In [60]:
import os

paths = ['c:/W_DATA_ROAM/ДОКУМЕНТАЦИЯ/NOC_HOW_IT_WORKS/',
        'c:/W_DATA_ROAM/МЕТОДИКИ/',
        'c:/W_DATA_ROAM/ПРОЕКТЫ/',
        'c:/W_DATA_ROAM/ДОКУМЕНТАЦИЯ/NOC_KNOWN_ISSUES/',
        'c:/W_DATA_ROAM/МЕТОДИКИ/_NOC_M_NEW_CONNECTION/',
        'c:/W_DATA_ROAM/ALARMS/',
        'c:/W_DATA_ROAM/ДОКУМЕНТАЦИЯ/PRICES',
        ]

for path in paths:
    names = os.listdir(path)
    [print(normpath(path + name)) for name in names if name.upper().find('DEUD2')!=-1]

c:\W_DATA_ROAM\ПРОЕКТЫ\190205_AL_Mon_Germ_add_DEUD2
c:\W_DATA_ROAM\ПРОЕКТЫ\190319_AL_Mondicon_DEUD2
c:\W_DATA_ROAM\ALARMS\MOC_Partner_DEUD2_#2031463_181114
c:\W_DATA_ROAM\ALARMS\PC_DEUD2_Partner_1092860_171218
c:\W_DATA_ROAM\ALARMS\PS_DEUD2_P4
c:\W_DATA_ROAM\ALARMS\PS_DEUD2_Partner_180508
c:\W_DATA_ROAM\ALARMS\TAP_DEUD2_Missing_for_Weconnect_RT


### Search in Allowed Lists and others

In [20]:
downloads = 'C:/Users/balob/Downloads'
dmi_allowed_list = 'DMI.dmi_allowed_list_export_Mon_Apr_01_2019.csv'
dmi_netpfx = 'DMI.dmi_netpfx_export_Wed_Mar_20_2019.csv'
bss_usage = 'bss_usage_2019_03.csv'

#Like expression for Allowed List
df_al=pd.read_csv(join(downloads, dmi_allowed_list))
df_al['SPONSOR'] = df_al['Sponsor'].replace(to_replace={r'[Pp][Aa][rtner]?[\s\w-]*':'S1', r'[Pp]4[\s\w-]*':'S2'}, regex=True)
df_al.rename({'PLMN code':'TADIG'}, axis=1, inplace=True)
#df_al_lab=pd.read_csv('LAB_DMI.dmi_allowed_list_export_Wed_Oct_03_2018.csv')

df_usage = pd.read_csv(join(downloads, bss_usage), sep=';')
cols = ['REP_DATE','VPLMN','SPONSOR','MOC','MOC_SMS','DATA']
df_usage = df_usage.pivot_table(values='COST', index=['REP_DATE', 'VPLMN', 'SPONSOR'], columns='USAGE_TYPE').reset_index()[cols]
df_usage.rename({'VPLMN':'TADIG'}, axis=1, inplace=True)
df_usage.columns.name=''

df_al = pd.merge(df_al, df_usage, how='left', on=['SPONSOR','TADIG'])

#df_dmi_diam_oper=pd.read_csv(join(download,''))
df_np=pd.read_csv(join(downloads, dmi_netpfx))
#df_did=pd.read_csv('HRR.hrr_did_export_Wed_Aug_08_2018.csv')
#f_tap_ocs=pd.read_csv('tap_ocs_analysis_180817.csv')

In [37]:
sponsor = 'P4' # Use regular expressions if it's needed
tadig = 'ISR'

df_al.loc[(df_al['TADIG'].str.startswith(tadig, na=False)) &
          (df_al['Sponsor'].str.contains(sponsor, na=False, regex=True, flags=re.I)),
          ['Sponsor','TADIG','REP_DATE','MOC','MOC_SMS','DATA']].sort_values(by=['Sponsor','TADIG'])

,Sponsor,TADIG,REP_DATE,MOC,MOC_SMS,DATA
20,P4,ISR01,2019-03,30.863655,10.025884,NaN
1065,P4_2018,ISR01,2019-03,30.863655,10.025884,NaN
1066,P4_2018,ISRPL,2019-03,NaN,NaN,1.050838
2515,P4_All,ISR01,2019-03,30.863655,10.025884,NaN
2516,P4_All,ISRPL,2019-03,NaN,NaN,1.050838
939,P4_All Rated,ISR01,2019-03,30.863655,10.025884,NaN
940,P4_All Rated,ISRPL,2019-03,NaN,NaN,1.050838
2794,P4_Clay_Combined,ISR01,2019-03,30.863655,10.025884,NaN
2225,P4_Combined,ISR01,2019-03,30.863655,10.025884,NaN
1782,P4_ISRPL,ISRPL,2019-03,NaN,NaN,1.050838


In [24]:
df_np[(df_np.Operator.str.startswith('CODCT',na=False))] #.to_csv(join(downloads,'Prefix.csv')) # & (df_np.SSN==7)

,Operator,Prefix,Number/Range,SSN,MCC,MNC
3168,CODCT,2439961,1,7,630,2
3169,CODCT,2439961,1,149,630,2
8443,CODCT,2439961004,0,149,630,2
8444,CODCT,2439961008,0,149,630,2
9744,CODCT,24399610040,0,1,630,2


In [32]:
plmn = 'CODVC'
df1 = df_al.loc[(df_al['TADIG'].str.startswith(plmn, na=False)) &
          (df_al['Sponsor'].str.contains('', na=False)), ['Sponsor','TADIG']].sort_values(by=['Sponsor','TADIG'])
df2 = df_np[(df_np.Operator.str.startswith(plmn,na=False))] #.to_csv(join(downloads,'Prefix.csv')) # & (df_np.SSN==7)
display(df1)
display(df2)

,Sponsor,TADIG
1815,Partner_Clay_Com,CODVC
3323,Partner_Combined,CODVC
679,Partner_STI_Comb,CODVC


,Operator,Prefix,Number/Range,SSN,MCC,MNC
840,CODVC,24381,1,7,630,1
841,CODVC,24381,1,149,630,1
5140,CODVC,2438181959,0,7,630,1
8866,CODVC,2438181999,0,149,630,1
9740,CODVC,24381819990,0,1,630,1


In [442]:
str(df_np[(df_np.Prefix.str.startswith('6683',na=False))].Prefix.unique().tolist())
df_np[(df_np.Prefix.str.startswith('919822',na=False))]

,Operator,Prefix,Number/Range,SSN,MCC,MNC
2642,INDBO,919822,1,7,404,22
2643,INDBO,919822,1,149,404,22


### Hex to Dec conversion

In [130]:
hex_str = '1172'
dec_str = '2505'
dec_int = 1962
bin_str = '1010'

# Hex to dec
print('Hex to dec')
print(int(hex_str,16))

# Dec to hex
print('Dec to hex')
print(hex(dec_int).upper())
print('%x'.upper() % dec_int)

# Dec to bin
print('Dec to bin')
print(bin(dec_int)[2:].zfill(8))

# Bin to dec
print('Bin to dec')
print(int(bin_str,2))

# Hex to bin
print('Hex to bin')
print(bin(int(hex_str, 16))[2:].zfill(8))
print(bin(0xABC123EFFF))
f'{0xAB:0>16b}'

Hex to dec
4466
Dec to hex
0X7AA
7AA
Dec to bin
11110101010
Bin to dec
10
Hex to bin
1000101110010
0b1010101111000001001000111110111111111111


'0000000010101011'

In [188]:
f'{bin(2)} - {bin(61)} - {bin(6)}'
#2.61.6

'0b10 - 0b111101 - 0b110'

In [186]:
print(bin(10915))
a = '10101010100011'
# = '00000011001101'
print(int(a[-14:-11],2),'-',int(a[-11:-3],2),'-',int(a[-3:],2) )

0b10101010100011
5 - 84 - 3


### Rename files

In [ ]:
import os
import re
path = 'c:/Users/balob/Downloads/new1/'
prefix = 'PYCON'
topic = ''
os.chdir(path)
names = sorted(filter(os.path.isfile, os.listdir('.')), key=os.path.getmtime)
for i,name in enumerate(names):
    short_name = re.sub('[!#?«»,() \+\-؟]','',name[:-4:][:50])
    if i < 10:
        number = '0'+str(i)
    else:
        number = str(i)
    new_name = '{}_{}_{}.mp4'.format(prefix,number,short_name)
    print('{} -> {}'.format(name,new_name))
    os.rename(name,new_name)

### Чтение логов с удаленного сервера SFTP (stat_GrayLog.log)

In [86]:
import paramiko
print_lines = 50
host = '172.19.11.191'
user = 'pavel'
secret = 'RzAZPjMjvujCPjU88bpevQp'
file_path_name = '/home/tracer/LOG/stat_GrayLog.log'
port = 22
client = paramiko.SSHClient()
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
client.connect(hostname=host, username=user, password=secret, port=port)
sftp_client = client.open_sftp()
remote_file = sftp_client.open(file_path_name)
try:
    #for line in remote_file:
    for i,line in enumerate(reversed(list(remote_file))):
        if i < print_lines:
            if 'Total' in line:
                print(i,line,end='')
            #print(i,line,end='')
        else:
            break
finally:
    remote_file.close()
client.close()
# (https://stackoverflow.com/questions/1596963/read-a-file-from-server-with-ssh-using-python)

0 2019-01-24 05:00:45,431 - ss7_stat - INFO - rep_ss7_sqlite_db_py2. End. Total elapsed time 0:00:43.978321
8 2019-01-23 05:00:47,552 - ss7_stat - INFO - rep_ss7_sqlite_db_py2. End. Total elapsed time 0:00:45.941926
16 2019-01-22 05:00:49,749 - ss7_stat - INFO - rep_ss7_sqlite_db_py2. End. Total elapsed time 0:00:45.060281
24 2019-01-21 05:00:49,224 - ss7_stat - INFO - rep_ss7_sqlite_db_py2. End. Total elapsed time 0:00:44.623832
32 2019-01-20 05:00:47,066 - ss7_stat - INFO - rep_ss7_sqlite_db_py2. End. Total elapsed time 0:00:45.239989
40 2019-01-19 05:00:45,864 - ss7_stat - INFO - rep_ss7_sqlite_db_py2. End. Total elapsed time 0:00:43.761820
48 2019-01-18 05:00:46,195 - ss7_stat - INFO - rep_ss7_sqlite_db_py2. End. Total elapsed time 0:00:43.864928


### Pricing counting

In [192]:
moc=1.5
mtc=0
text=0.25
data=7.37

tariff=[moc,mtc,text,data]

print("Price P4")
for k,v in {'1.2':1.2,'1.2*1.5':1.2*1.5,'1.2*1.5*1.15':1.2*1.5*1.15,'1.2*1.5*1.30':1.2*1.5*1.30,'2':1.2*2}.items():
    print("Price {:<15}:".format(k),[round(i*v,6) for i in tariff])
print("\nPrice Partner")
for k,v in {'1.5':1.5,'1.5*1.15':1.5*1.15,'1.5*1.30':1.5*1.30,'2':2}.items():
    print("Price {:<15}:".format(k),[round(i*v,6) for i in tariff])

Price P4
Price 1.2            : [1.8, 0.0, 0.3, 8.844]
Price 1.2*1.5        : [2.7, 0.0, 0.45, 13.266]
Price 1.2*1.5*1.15   : [3.105, 0.0, 0.5175, 15.2559]
Price 1.2*1.5*1.30   : [3.51, 0.0, 0.585, 17.2458]
Price 2              : [3.6, 0.0, 0.6, 17.688]

Price Partner
Price 1.5            : [2.25, 0.0, 0.375, 11.055]
Price 1.5*1.15       : [2.5875, 0.0, 0.43125, 12.71325]
Price 1.5*1.30       : [2.925, 0.0, 0.4875, 14.3715]
Price 2              : [3.0, 0, 0.5, 14.74]


## Testing

In [ ]:
import requests
#payload = {'key1':'value1','key2':'value2'}
#r = requests.get('http://192.168.1.44:80/climate;sensor=1&readattempt=10&temperature=nan&humidity=nan')
r = requests.get('http://192.168.1.44:80/climate;sensor=1&readattempt=10&temperature=10.0&humidity=10.0')
#r = requests.get('http://192.168.1.53:80/connected;IpAddr=192.168.1.63')
print('Status: ',r.status_code,'\n',r.text)
r.close()

In [513]:
charge = 0.02776
exchangeRate = 1.22876
sdrRate = 1.39158
(charge * sdrRate)
# 0.0386302608

0.0386302608

In [514]:
charge = 0.10904
exchangeRate = 1.22876
sdrRate = 1.39158
charge * sdrRate

0.1517378832

In [677]:
charge = 0.19257
costTariff = 0.00105
sdrRate = 1.2
costTariff * sdrRate

0.0012599999999999998

In [1]:
charge = 0.04807
chargeableUnits = 5007360
exchangeRate = 1.22876
dataRateCombinedCharge2019 = 0.0028 # EUR
(chargeableUnits * dataRateCombinedCharge2019) / 1024 / 1024 #0.26797155 EUR
(charge / chargeableUnits) * 1024 * 1024

0.010066192229038855

In [10]:
14938.91 * 1.31

19569.9721

In [12]:
14938.91 * 1.13 = 16880.96

16880.968299999997

In [153]:
5 * 5

25

In [ ]:
#simple_server.py
import socket
host = ""
port = 9999
s = socket.socket()
s.bind((host, port))
s.listen(5) # where 5 is number of incoming connections
conn, address = s.accept()
print("Connection has been established! |" + " IP " + address[0] + " | Port" + str(address[1]))
conn.send(str.encode('Hello world from Server!'))
client_response = str(conn.recv(1024),"ISO-8859-1")
print(client_response, end="")
conn.close()

In [136]:
#server.py

import socket
import sys

host = ""
port = 9999

# Create a Socket ( connect two computers)
def create_socket():
    try:
        s = socket.socket()
        return s
    except socket.error as msg:
        print("Socket creation error: " + str(msg))


# Binding the socket and listening for connections
def bind_socket(socket):
    try:
        print("Binding the Port: " + str(port))
        socket.bind((host, port))
        socket.listen(5)
    except socket.error as msg:
        print("Socket Binding error" + str(msg) + "\n" + "Retrying...")

# Establish connection with a client (socket must be listening)

def socket_accept(socket):
    conn, address = socket.accept()
    print("Connection has been established! |" + " IP " + address[0] + " | Port" + str(address[1]))
    send_commands(conn)
    conn.close()

# Send commands to client/victim or a friend
def send_commands(conn):
    while True:
        cmd = input()
        if cmd == 'quit':
            conn.close()
            s.close()
            sys.exit()
        if len(str.encode(cmd)) > 0:
            conn.send(str.encode(cmd))
            client_response = str(conn.recv(1024),"ISO-8859-1")
            print(client_response, end="")

def main():
    s = create_socket()
    bind_socket(s)
    socket_accept(s)


main()

Say:
You said: hello


In [79]:
import shutil
shutil.copy(join(dir_david, file_sqlite_db), join(dir_david, 'david_db.sqlite_190412'))

'c:\\Users\\balob\\Downloads\\DAVID\\david_db.sqlite_190412'

In [76]:
import datetime as dt

f'david_db_{dt.datetime.now().strftime("%Y%m%d")}.sqlite'

'david_db_20190412.sqlite'

In [85]:
import unittest
import requests
from os.path import isfile, isdir, join
import datetime as dt
import shutil

import david_lib
reload(david_lib)

server_ip_addr = david_lib.ip_addr
server_port = david_lib.port
dir_david = david_lib.dir_david
file_climate_hot_bedroom = david_lib.file_climate_hot_bedroom
file_sqlite_db = david_lib.file_sqlite_db
file_log_web_server = david_lib.file_log_web_server
file_log_climate_check = david_lib.file_log_climate_check

file_sqlite_db_backup = f'david_db_{dt.datetime.now().strftime("%Y%m%d")}.sqlite'

class TestFiles(unittest.TestCase):
    def test_check_file(self):
        self.assertEqual(isdir(dir_david), True)
        self.assertEqual(isfile(join(dir_david, file_climate_hot_bedroom)), True)
        self.assertEqual(isfile(join(dir_david, file_sqlite_db)), True)
        self.assertEqual(isfile(join(dir_david, file_log_web_server)), True)
        self.assertEqual(isfile(join(dir_david, file_log_climate_check)), True)


class TestWebServer(unittest.TestCase):
    
    @classmethod
    def setUpClass(cls):
        shutil.copy(join(dir_david, file_sqlite_db), join(dir_david, file_sqlite_db_backup))
        
    @classmethod
    def tearDownClass(cls):
        shutil.copy(join(dir_david, file_sqlite_db_backup), join(dir_david, file_sqlite_db))
    
    def test_get_connect(self):
        url_01 = f'http://{server_ip_addr}:{server_port}/connected;sensor=1&ip=192.168.1.63'
        url_02 = f'http://{server_ip_addr}:{server_port}/connected;sensor=3&ip=192.168.1.64'
        urls = [url_01, url_02]
        for url in urls:
            r = requests.get(url)
            self.assertEqual(r.status_code, 200)
            self.assertEqual(r.text, 'OK')
        
    def test_get_climate(self):
        url_01 = f'http://{server_ip_addr}:{server_port}/climate;sensor=1&readattempt=10&temperature=24.0&humidity=35.0'
        url_02 = f'http://{server_ip_addr}:{server_port}/climate;sensor=1&readattempt=10&temperature=nan&humidity=35.0'
        url_03 = f'http://{server_ip_addr}:{server_port}/motion;sensor=3'
        urls = [url_01, url_02, url_03]
        for url in urls:
            r = requests.get(url)
            self.assertEqual(r.status_code, 200)
            self.assertEqual(r.text, 'OK')

            
if __name__ == '__main__':
    unittest.main()


#suite = unittest.TestLoader().loadTestsFromTestCase(TestFiles)
#runner = unittest.TextTestRunner(verbosity=2)
#runner.run(suite)

#suite = unittest.TestLoader().loadTestsFromTestCase(TestWebServer)
#runner = unittest.TextTestRunner(verbosity=2)
#runner.run(suite)

test_check_file (__main__.TestFiles) ... FAIL

FAIL: test_check_file (__main__.TestFiles)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-85-56cc4e7100b5>", line 22, in test_check_file
    self.assertEqual(isdir(dir_david), True)
AssertionError: False != True

----------------------------------------------------------------------
Ran 1 test in 0.004s

FAILED (failures=1)
ERROR

ERROR: setUpClass (__main__.TestWebServer)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-85-56cc4e7100b5>", line 33, in setUpClass
    shutil.copy(join(dir_david, file_sqlite_db), join(dir_david, file_sqlite_db_backup))
  File "C:\Users\balob\Miniconda3\lib\shutil.py", line 241, in copy
    copyfile(src, dst, follow_symlinks=follow_symlinks)
  File "C:\Users\balob\Miniconda3\lib\shutil.py", line 120, in copyfile
    with open(src, 'rb') as fsrc:
FileN

<unittest.runner.TextTestResult run=0 errors=1 failures=0>

In [208]:
%reset -f